In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
import datetime
import pandas as pd
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset

In [14]:
import importlib.util
import sys

# Specify the full path to the module file
module_path = 'D:\\Github Folder\\MasterThesis\\notebooks\\FUNCTIONS.py'

# Load the module
spec = importlib.util.spec_from_file_location("FUNCTIONS", module_path)
functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(functions)

# Now you can use the functions as if you had imported them
load_datasets = functions.load_datasets
create_dataloaders = functions.create_dataloaders
train_and_validate = functions.train_and_validate
test_model = functions.test_model


In [15]:
df = pd.read_excel("references\Subject_info_balanced.xlsx")

In [16]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}

In [17]:
# Model definition
class Baseline3DCNN(nn.Module):
    def __init__(self, num_classes=2, init_filters=32, kernel_size=3, stride=2, num_fc_units=128):
        super(Baseline3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, init_filters, kernel_size=kernel_size, stride=stride, padding=1)
        self.conv2 = nn.Conv3d(init_filters, init_filters*2, kernel_size=kernel_size, stride=stride, padding=1)
        self.conv3 = nn.Conv3d(init_filters*2, init_filters*4, kernel_size=kernel_size, stride=stride, padding=1)
        self.pool = nn.MaxPool3d(2)
        self.relu = nn.ReLU()

        # Compute the flattened size after all convolutions and pooling
        self.final_dim = self._get_conv_output_dim(193, 3, stride, kernel_size, init_filters*4)
        self.fc1 = nn.Linear(self.final_dim, num_fc_units)
        self.fc2 = nn.Linear(num_fc_units, num_classes)

    def _get_conv_output_dim(self, input_dim, num_convs, stride, kernel_size, num_filters):
        output_dim = input_dim
        for _ in range(num_convs):
            output_dim = ((output_dim - kernel_size + 2 * (kernel_size // 2)) // stride + 1) // 2  # Pooling divides size by 2
        return output_dim * output_dim * output_dim * num_filters

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [18]:
from openpyxl import load_workbook
import pandas as pd
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim


def run_experiment(df, config):
    """Run the experiment with the given configuration on the preprocessed DataFrame."""
    train_dataset, val_dataset, test_dataset = load_datasets(df, config['image_type'])
    train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config['batch_size'])
    
    # Initialize model and training components
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Baseline3DCNN(num_classes=config['num_classes'], init_filters=config['init_filters'],
                          kernel_size=config['kernel_size'], stride=config['stride'], num_fc_units=config['num_fc_units']).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    # Training and validation
    train_accuracies, val_accuracies, val_losses = train_and_validate(model, train_loader, val_loader, criterion, optimizer, label_mapping, config['num_epochs'], config['patience'], device)
    test_results, test_accuracy = test_model(model, test_loader, label_mapping, device)
    
    
    
    
  # Save detailed results to Excel
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')
    filename = os.path.join('reports', f'{formatted_time}_Experiment.xlsx')
    
    summary_data = {
        'Phase': ['Training', 'Validation', 'Testing'],
        'Accuracy': [train_accuracies[-1], val_accuracies[-1], test_accuracy]
    }
    summary_df = pd.DataFrame(summary_data)
    all_results = pd.DataFrame(test_results)
    config_df = pd.DataFrame([config])
    
    with pd.ExcelWriter(filename) as writer:
        config_df.to_excel(writer, sheet_name='Configuration')
        all_results.to_excel(writer, sheet_name='Results')
        summary_df.to_excel(writer, sheet_name='Summary')

    # Append a summary of this experiment to the cumulative RESULTS.xlsx file
    results_file = os.path.join('reports', 'RESULTS.xlsx')
    experiment_summary = {**config, **{'Training Accuracy': train_accuracies[-1], 'Validation Accuracy': val_accuracies[-1], 'Test Accuracy': test_accuracy, 'DATETIME': formatted_time}}
    summary_row = pd.DataFrame([experiment_summary])

    if os.path.exists(results_file):
        with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            existing_df = pd.read_excel(results_file)
            # Reindex the existing DataFrame to ensure all columns are aligned and add new columns if necessary
            combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
            combined_df = combined_df.reindex(columns=(existing_df.columns.tolist() + [col for col in summary_row.columns if col not in existing_df.columns]))
            combined_df.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        summary_row.to_excel(results_file, index=False)

    return filename, train_accuracies[-1], val_accuracies[-1], test_accuracy

# Example configuration and use case
config = {
    'num_classes': 2,
    'init_filters': 128,
    'kernel_size': 3,
    'stride': 2,
    'num_fc_units': 128,
    'optimizer': 'Adam',
    'loss_criterion': 'BCEWithLogitsLoss',
    'num_epochs': 2,
    'batch_size': 1,
    'patience': 3
}

In [19]:
image_types = ['MRI_PET', 'MRI', 'PET']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)


Working on image type: MRI_PET


Epoch 1/2 - Train: 100%|██████████| 76/76 [00:13<00:00,  5.68it/s]


Epoch 1: Train Loss: 2.4332 - Train Accuracy: 50.00%


Epoch 1/2 - Validate: 100%|██████████| 26/26 [00:03<00:00,  7.72it/s]


Epoch 1: Validation Loss: 0.6937 - Validation Accuracy: 50.00%


...

Epoch 2/2 - Train: 100%|██████████| 76/76 [00:10<00:00,  7.00it/s]


Epoch 2: Train Loss: 0.6938 - Train Accuracy: 39.47%


Epoch 2/2 - Validate: 100%|██████████| 26/26 [00:03<00:00,  7.90it/s]


Epoch 2: Validation Loss: 0.6932 - Validation Accuracy: 50.00%


Testing: 100%|██████████| 26/26 [00:03<00:00,  7.33it/s]
